# Instructions to reproduce the results on GCP

### GPU:

- Go to [AI platform notebooks](https://cloud.google.com/ai-platform-notebooks/)
- Press "Go to console". If you don't see "Go to console" you need to create
  a GCP account for free:
    -  Press "Get started for free" and create an account.
    - Open [AI platform notebooks](https://cloud.google.com/ai-platform-notebooks/)
    - Press "Go to console"
    - Press "Enable API"
    - Wait for the previous step to finish and press "GO TO INSTANCES PAGE"
- Press "New Insance" at the top -> "Customize instance".
- Create a new instance with the following specifications:
    - Region: **us-central1**
    - Zone: **us-central1-b** (this is important for TPU)
    - Environment: TensorFlow 2.1 Enterprise
    - Machine type: `n1-highcpu-96` (96 vCPUs, 86.4 GB RAM)
    - GPU Type: NVIDIA TESLA v100 GPU
    - Tick "install NVIDIA GPU driver"
    - Press create
    - Press JUPYTERLAB and upload this notebook


### TPU:
- Follow the steps from from above but the GPU set up may be omitted
- Go to [Compute Engine -> TPUs](https://pantheon.corp.google.com/compute/tpus)
- Follow the hints to create a TPU node with the following specs:
     -  Zone: **us-central1-b**
     - TPU type: `v2-8` (you can use `v3-8` for better performance)
     - TPU software version: `nightly`
     - Press "create"
- Wait for the node to start up. Take a note of the internal IP (something like `10.245.84.146`)
- Open the JUPYTERLAB created by the steps above.




# Notes

In this colab we perform CVA-like calculation for a batch of vanilla interest 
rate swaps. We assume an underlying Hull-White model for a short rate.
We propagate the rate for 136 steps and price 1 million swaps at each iteration.
The swaps have tenures of up to 30 years with payment frequencies varying from
1 to 12 months. 

The whole procedure takes **3 seconds** on a **TPU**. This is currently done in a single precision but should be shortly be available in double precision too. 

We compare sampling speed against CPU and GPU and provide a reference to QuantLib sampling speed.


# Hull White future yield curves.

For the single factor Hull white model, the conditional forward bond prices are of the [affine form](https://en.wikipedia.org/wiki/Hull%E2%80%93White_model#Bond_pricing_using_the_Hull%E2%80%93White_model):

$$P(S, T) = A(S, T) e^{-B(S,T) r(S)}$$

where

$$B(S, T) = \frac{1}{\alpha} \left(1 - e^{-\alpha (T-S) } \right)$$

and

$$\begin{eqnarray}
\ln A(S, T) &=& \ln \frac{P(0, T)}{P(0, S)} + B(S, T) f(0, S) - \frac{\sigma^2}{4\alpha^3}\left[1-e^{-\alpha (T-S)}\right]^2 (1-e^{-2\alpha S}) \\
&=& \ln \frac{P(0, T)}{P(0, S)} + B(S, T) f(0, S) - \frac{\sigma^2}{4\alpha}B(S,T)^2 (1-e^{-2\alpha S})
\end{eqnarray}
$$

and
$$f(0, S) = -\frac{\partial}{\partial S} \ln P(0, S)$$

Assuming we are given the pair $(S, r(S))$, we can use the above to compute the discount factors as "observed" at time $S$. The set of future times will be given to us and the $r$ at those times will be computed by sampling (see next section.).t those times will be computed by sampling (see next section.).

In [1]:
#@title Imports
import numpy as np
import tensorflow.compat.v2 as tf
import time

# Disable eager execution since TPU routines are better handled in graph mode
tf.compat.v1.disable_eager_execution()

import tf_quant_finance as tff

# Load TFF dates library
dates = tff.experimental.dates

In [2]:
#@title Global dtype. TPU will soon support FP64
dtype = np.float32 #@param

In [3]:
#@title TFF Fixing funtion and VanillaSwap class

def get_fixings(*, dates_tensor, discount_fn, day_count, dtype):
  """Computes fixings implied by the input dates tensor and a discounting curve.

  Given dates `[d1, d2, .. , dn]` computes forward rates `fwd_rates` between
  `[d_i, d_{i+1}]` for `i=0,..., n-1` and returns the corresponding deposit
  rates defined as `1 + deposit_rates * day_count(d_i, d_{i+1}) = fwd_rates`.

  Args:
    dates_tensor: `DateTensor` of shape `[num_dates, batch_shape]`.
    discount_fn: A callable that maps `DateTensor` to a real number of `dtype`
      which corresponds to discounting.
    day_count: A daycounting function.
    dtype: Output `dtype`.
  
  Returns:
    A `Tensor` of the specified `dtype` and of shape
    `[num_dates - 1, batch_shape]` that correponds to the deposit rates at
    `dates_tensor[1:]`.
  """ 
  start_dates = dates_tensor[:-1]
  end_dates = dates_tensor[1:]
  disc_start = discount_fn(start_dates)
  disc_end = discount_fn(end_dates)
  t = day_count(start_date=start_dates, end_date=end_dates, dtype=dtype)
  if t.shape.as_list() != disc_end.shape.as_list():
    t = tf.expand_dims(t, axis=-1)
  fixings = (disc_start/disc_end - 1.0) / t
  fixings = tf.where(t > 0, fixings, 0)
  return fixings

class VanillaSwap:
  """Simple interest rate swap."""
  def __init__(self,
               *,
               calc_date,
               fixed_leg_dates,
               float_leg_dates,
               fixed_leg_rates,
               float_leg_rates,
               notional,
               day_count,
               discount_fn,
               dtype=None):
    """Initializer.
    
    Args:
      calc_date: An instance of `DateTensor` of zero shape. The reference date
        to which perform the discounting.
      fixed_leg_dates: A `DateTensor` of shape `[batch_shape, n]` representing
        the cashflow dates of the fixed leg including the `calc_date` as the
        first entry for each swap in the batch.
      float_leg_dates: A `DateTensor` of shape `[batch_shape, n]` representing
        the cashflow dates of the float leg including the `calc_date` as the
        first entry for each swap in the batch. 
      fixed_leg_rates: A real `Tensor` of shape brodcastable with
        `[batch_shape, n]` representing the fixed rates of the swap.
      float_leg_rates: A real `Tensor` of shape brodcastable with
        `[batch_shape, n]` and of the same dtype as `fixed_leg_rates`.
        Represents the float rates of the swap.
      notional: A real `Tensor` of shape brodcastable with `[batch_shape, n]`
        and of the same dtype as `fixed_leg_rates`. Represents the notional of
        the swap.
      day_count: A daycount convention. One of `dates.daycounts`.
      discount_fn: A callable that maps `DateTensor` to a real number of the
        same `dtype` as `fixed_leg_rates` which corresponds to the discounting
        function.
      dtype: A `dtype` for the underlying real `Tensor`s.
        Default value: None which maps to the `dtype` inferred by TensorFlow.
    """  
    self._calc_date = calc_date
    self._fixed_leg_dates= fixed_leg_dates
    self._float_leg_dates = float_leg_dates
    self._fixed_leg_rates = fixed_leg_rates
    self._float_leg_rates = float_leg_rates
    self._notional = notional
    self._day_count = day_count
    self._discount_fn = discount_fn
    self._dtype = dtype

  def fixed_cashflows(self):
    """Returns all fixed cashflows at `fixed_leg_dates`."""
    start_dates = self._fixed_leg_dates[:-1]
    end_dates = self._fixed_leg_dates[1:]
    t = self._day_count(
        start_date=start_dates, end_date=end_dates, dtype=self._dtype)
    t = tf.nn.relu(t)
    if t.shape.as_list() != self._float_leg_rates.shape.as_list():
      t = tf.expand_dims(t, axis=-1)
    return self._notional * self._fixed_leg_rates * t

  def float_cashflows(self):
    """Returns all float cashflows at `float_leg_dates`."""
    start_dates = self._float_leg_dates[:-1]
    end_dates = self._float_leg_dates[1:]
    t = self._day_count(
        start_date=start_dates, end_date=end_dates, dtype=self._dtype)
    t = tf.nn.relu(t)
    if t.shape.as_list() != self._float_leg_rates.shape.as_list():
      t = tf.expand_dims(t, axis=-1)
    return self._notional * self._float_leg_rates * t

  def float_leg_present_value(self):
    """Returns the value of the float leg discounted to `self.calc_date`."""
    payment_dates = self._float_leg_dates[1:]
    cashflows = self.float_cashflows()
    return tf.reduce_sum(cashflows * self._discount_fn(payment_dates),
                         axis=0)

  def fixed_leg_present_value(self):
    """Returns the value of the fixed leg discounted to `self.calc_date`."""
    payment_dates = self._fixed_leg_dates[1:]
    cashflows = self.fixed_cashflows()
    return tf.reduce_sum(cashflows * self._discount_fn(payment_dates),
                         axis=0)

  def price(self):
    """Returns the value of the swap discounted to `self.calc_date`."""
    return self.float_leg_present_value() - self.fixed_leg_present_value()


In [4]:
#@title Swap schedule generation and Hull-White model parameters

from collections import namedtuple

YieldParams = namedtuple('YieldParams', ['a0', 'a1', 'a2'])

def random_yield_params(size, max_time=30.0):
  r0 = dtype(np.random.rand(size)* (0.07 - 0.005) + 0.005)
  rT = dtype(np.random.rand(size)* (0.07 - 0.005) + 0.005)
  r_mins = np.minimum(r0, rT)
  r_maxs = np.maximum(r0, rT)
  do_low = np.random.rand(size) > dtype(0.5)
  a0 = np.random.rand(size)
  a0 = np.where(do_low, a0 * (r_mins - 0.0001) + 0.0001, a0 * (0.08 - r_maxs) + r_maxs)
  a2 = max_time / (1 + np.random.choice([-1.0, 1.0], size=size) * np.sqrt((rT - a0)/(r0-a0)))
  a1 = (r0 - a0) / a2 / a2
  return YieldParams(a0=a0,a1=a1,a2=a2)

def log_current_discount_fwd_fn(yield_params):
  """Suitable for the next log discount evaluator below."""
  a0 = np.array(yield_params.a0, dtype=dtype)
  a1 = np.array(yield_params.a1, dtype=dtype)
  a2 = np.array(yield_params.a2, dtype=dtype)
  def eval_fn(times):
    """Gives the log zero coupon bond price and the instantaneous forward rate."""
    return -(a0 + a1 * (times - a2)**2) * times, (a0 - a1 * a2 * a2 / 3) + 3 * a1 * (times - 2 * a2 / 3) ** 2
  return eval_fn


HullWhiteData = namedtuple('HullWhiteData',
                           ['mean_reversion', 'volatility',
                            'log_discount_fwd_fn'])


def gen_hull_white_params():
  mean_reversion = np.random.rand() * 0.1
  volatility = np.random.rand() * 0.3
  present_yield_curve_params = random_yield_params(1)
  return HullWhiteData(
      mean_reversion=mean_reversion,
      volatility=volatility,
      log_discount_fwd_fn=log_current_discount_fwd_fn(present_yield_curve_params))
  
def generate_short_rates(initial_short_rates,
                         hull_white_params, times, num_scenarios,
                         dtype):
  a = dtype(hull_white_params.mean_reversion)
  sigma = dtype(hull_white_params.volatility)
  def instant_forward_rate_fn(t):
    return hull_white_params.log_discount_fwd_fn(t)[1]
  process = tff.models.hull_white.HullWhiteModel1F(
      mean_reversion=a, volatility=sigma,
      instant_forward_rate_fn=instant_forward_rate_fn,
      dtype=dtype)
  sample_paths = process.sample_paths
  paths = sample_paths(
      times,
      num_samples=num_scenarios,
      initial_state=initial_short_rates,
      seed=42)
  paths = tf.squeeze(paths, axis=-1)
  # Shape [num_times, num_samples]
  return tf.transpose(paths)

def discount_curve_at_times(
    calc_date,
    short_rates,
    start_date,
    end_date,
    current_disc_fwd_fn,
    day_count,
    mean_revs,
    sigmas):
  """Computes forward discount factors.

  Produces P(S, T) i.e. the discount factor as seen at time 'S' the calculation date
  for expiry at time 'T' the evaluation date.

  Time today is 0. The eval times are allowed to be negative but not the calc times

  Args:
    calc_date: current date.
    short_rates: The short rates of shape: [N_scenarios].
    start_date: The calculation time.
    end_date: The evaluation dates.
    currency_disc_fwd_fn: A callable that returns instantaneous forward rate and
      the log-discount at the specified times. 
    mean_revs: A tensor of shape [num_currencies] The mean reversions for
      the HW model.
    sigmas: A tensor of same shape as mean_revs.
  
  Returns: A tensor of shape [end_date.shape] + [N_scenarios]
  """
  # S- T
  day_fractions = day_count(
      start_date=start_date,
      end_date=end_date, dtype=dtype)
  S = day_count(
      start_date=calc_date,
      end_date=start_date, dtype=dtype)
  T = day_count(
      start_date=calc_date,
      end_date=end_date, dtype=dtype)

  b_exp = mean_revs * day_fractions  # shape [N_calc_dates, n_eval_dates]
  b = (1 - tf.exp(-b_exp)) / mean_revs
  lnP_p = current_disc_fwd_fn(T)[0]  # shape [n_eval_dates]
  lnP_den, inst_fwd = current_disc_fwd_fn(S)  # output of shapes [n_calc_dates, n_eval_dates]
  lnA = (lnP_p - lnP_den + b * inst_fwd
         - ((sigmas * b) ** 2) * (1 - tf.exp(-2 * mean_revs * S)))
  lnA = tf.expand_dims(lnA, axis=-1)
  b = tf.expand_dims(b, axis=-1)
  short_rates = tf.expand_dims(short_rates, axis=0)
  discounts = tf.exp(lnA - b * short_rates)
  # Adjust for when calc_date > eval_date
  return discounts #tf.where(tf.expand_dims(T > S, -1), discounts, 1.0)


# Pricing vanilla
 swaps comparison. CPU vs GPU vs TPU

Note that **QuantLib** pricing speed for a swap with 40 payments:
**10000 swaps / sec** on a Intel . TPU can price **2 million / sec**

In [5]:
tf.compat.v1.reset_default_graph()
hull_white_params = gen_hull_white_params()
calc_date = dates.from_year_month_day(2015, 9, 9)
# Corresponding ql.WeekendsOnly calendar
calendar = dates.create_holiday_calendar(
    weekend_mask=dates.WeekendMask.SATURDAY_SUNDAY)
# Business day convention
bussiness_convention = dates.BusinessDayConvention.FOLLOWING
day_count = dates.daycounts.thirty_360_isda


In [6]:
# Generate swaps
# One tenor for simplicity
TENORS = [1, 3, 6, 12] # months

NUM_SWAPS = 100000 #@param

NUM_SWAPS_PER_TENOR = NUM_SWAPS // len(TENORS)
# Start date of the swaps

def generate_swaps():
    start_dates = []
    end_dates = []
    schedule_dates = []
    long_short = []
    for t in TENORS:
        random_shift = np.int32(t * 2 *
                              (np.random.rand(NUM_SWAPS_PER_TENOR) - 0.5) * 50)
        start_date = calendar.add_business_days(
          calc_date, random_shift, roll_convention=bussiness_convention)
        # Maximum swap tenure is 30 years
        swap_tenure = 1 + np.int32(30 * (np.random.rand(NUM_SWAPS_PER_TENOR)))
        period = dates.periods.PeriodTensor(swap_tenure, dates.PeriodType.YEAR)
        end_date = calendar.add_period_and_roll(start_date, period,
                                              bussiness_convention)
        # Exchange schedules
        schedule = dates.PeriodicSchedule(
          start_date=start_date, end_date=end_date,
          tenor=dates.periods.months(t),
          holiday_calendar=calendar,
          roll_convention=bussiness_convention)
        schedule_date = schedule.dates()
        # Record swap data
        start_dates.append(start_date)
        end_dates.append(end_date)
        schedule_dates.append(schedule_date.transpose())
        long_short.append(2 * (np.random.binomial(1, 0.48, size=random_shift.shape) - 0.5))
    return schedule_dates, long_short

In [7]:
# Precompute the schedules. This is done so that we measure only the pricing
# speed on a chosen platform
schedule_dates_day = []
schedule_dates_month = []
schedule_dates_year = []
schedule_dates, long_short = generate_swaps()
# Extract and precompute the schedules
for schedule in schedule_dates:
  schedule_dates_day.append(schedule.day())
  schedule_dates_month.append(schedule.month())
  schedule_dates_year.append(schedule.year())

sess = tf.compat.v1.Session()
#schedule_dates_tpu = sess.run(schedule_dates_tpu)
schedule_dates_day, schedule_dates_month, schedule_dates_year = sess.run(
    [schedule_dates_day, schedule_dates_month, schedule_dates_year])
sess.close()

In [8]:
schedule_dates

[DateTensor: shape=(362, 25000),
 DateTensor: shape=(122, 25000),
 DateTensor: shape=(62, 25000),
 DateTensor: shape=(32, 25000)]

In [9]:
def test_fn(core_id, inputs):
    #schedule_dates_day, schedule_dates_month, schedule_dates_year = inputs
    day, month, year, short_rate = inputs
    swaps = []
    calc_date = dates.from_year_month_day(year[core_id],
                                          month[core_id],
                                          day[core_id])
    schedule_dates = []
    for year, month, day in zip(schedule_dates_year,
                                schedule_dates_month,
                                schedule_dates_day):
      schedule_dates.append(dates.from_year_month_day(year,
                                                      month,
                                                      day))
    def build_discount_curve(schedule):
      def discount_curve(d):
        return tf.squeeze(discount_curve_at_times(
            calc_date,
            short_rate[core_id],
            d,
            schedule[1:],
            hull_white_params.log_discount_fwd_fn,
            day_count,
            hull_white_params.mean_reversion,
            hull_white_params.volatility), -1)
      return discount_curve
    for schedule in schedule_dates:

      # Compute the float let rates
      float_leg_rates = get_fixings(
              dates_tensor=schedule,
              day_count=day_count,
              discount_fn=build_discount_curve(schedule),
              dtype=dtype)
      #return float_leg_rates
      swap = VanillaSwap(
          calc_date=calc_date,
          fixed_leg_dates=schedule,
          float_leg_dates=schedule,
          fixed_leg_rates=0.0039,
          float_leg_rates=float_leg_rates,
          notional=1000000,
          day_count=day_count,
          discount_fn=build_discount_curve(schedule),
          dtype=dtype)
      swaps.append(swap)
    prices = [position * swap.price()
          for swap, position in zip(swaps, long_short)]
    return prices

#### CPU performance

In [18]:
NUM_SCENARIOS = 1000 #@param
g = tf.Graph()

with g.as_default():
  days = tf.compat.v1.placeholder(tf.int32, shape=[1],
                                  name="days_cpu")
  months = tf.compat.v1.placeholder(tf.int32, shape=[1],
                                    name="months_cpu")
  years = tf.compat.v1.placeholder(tf.int32, shape=[1],
                                   name="years_cpu")
  def cond(iter_num, res):
    return iter_num < NUM_SCENARIOS
  def body(iter_num, res):
    # Generate short rates here
    short_rate = 0.02 + tf.random.uniform([1], dtype=dtype)
    test_cpu = tf.xla.experimental.compile(
        test_fn, [0, [days, months, years, short_rate]])
    test_cpu = tf.concat(test_cpu, axis=0)
    iter_num_float = tf.cast(iter_num, dtype=dtype)
    return iter_num + 1, (res * iter_num_float
                          + test_cpu) / (iter_num_float + 1)
  with tf.device("/cpu:0"):
    test_cpu_res = tf.while_loop(cond, body,
                                 (0, tf.zeros([NUM_SWAPS], dtype=dtype)))
  sess_cpu = tf.compat.v1.Session()

In [20]:
# Time after optimization (run the cell twice)
t = time.time()
with g.as_default():
  sess_cpu.run(test_cpu_res, feed_dict={
      "days_cpu:0": [9],
      "months_cpu:0": [9],
      "years_cpu:0": [2015]})
print("Wall time (CPU): ", time.time() - t)

Wall time (CPU):  67.58648920059204


#### GPU performance

In [21]:
NUM_SCENARIOS = 1000 #@param
g = tf.Graph()

with g.as_default():
  days = tf.compat.v1.placeholder(tf.int32, shape=[1],
                                  name="days_gpu")
  months = tf.compat.v1.placeholder(tf.int32, shape=[1],
                                    name="months_gpu")
  years = tf.compat.v1.placeholder(tf.int32, shape=[1],
                                   name="years_gpu")
  def cond(iter_num, res):
    return iter_num < NUM_SCENARIOS
  def body(iter_num, res):
    # Generate short rates here
    short_rate = 0.02 + tf.random.uniform([1], dtype=dtype)
    test_gpu = tf.xla.experimental.compile(
        test_fn, [0, [days, months, years, short_rate]])
    test_gpu = tf.concat(test_gpu, axis=0)
    iter_num_float = tf.cast(iter_num, dtype=dtype)
    return iter_num + 1, (res * iter_num_float
                          + test_gpu) / (iter_num_float + 1)
  with tf.device("/gpu:0"):
    test_gpu_res = tf.while_loop(cond, body,
                                 (0, tf.zeros([NUM_SWAPS], dtype=dtype)))
  sess_gpu = tf.compat.v1.Session()

In [23]:
# Time after optimization (run the cell twice)
t = time.time()
with g.as_default():
  sess_gpu.run(test_gpu_res, feed_dict={
      "days_gpu:0": [9], "months_gpu:0": [9],
      "years_gpu:0": [2015]})
print("Wall time (GPU): ", time.time() - t)

Wall time (GPU):  1.111879587173462


In [24]:
%%timeit -n 10
with g.as_default():
  sess_gpu.run(test_gpu_res, feed_dict={
      "days_gpu:0": [9], "months_gpu:0": [9],
      "years_gpu:0": [2015]})

1.1 s ± 390 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### TPU performance

In [15]:
NUM_TPU_CORES =  32 #@param
NUM_SCENARIOS= 20000 #@param

g = tf.Graph()
with g.as_default():
  days = tf.compat.v1.placeholder(tf.int32, shape=[NUM_TPU_CORES],
                                  name="days_tpu")
  months = tf.compat.v1.placeholder(tf.int32, shape=[NUM_TPU_CORES],
                                    name="months_tpu")
  years = tf.compat.v1.placeholder(tf.int32, shape=[NUM_TPU_CORES],
                                   name="years_tpu")
  def cond(iter_num, res):
    return iter_num < NUM_SCENARIOS
  def body(iter_num, res):
    short_rate = 0.02 + tf.random.uniform([NUM_TPU_CORES], dtype=dtype)
    test_tpu = tf.compat.v1.tpu.replicate(
        test_fn,
        inputs=[
                [core_id,
                  [days,
                    months,
                    years,
                    short_rate]
                  ]
                for core_id in range(NUM_TPU_CORES)])
    test_tpu = [tf.concat(t, axis=0) for t in test_tpu]
    test_tpu = tf.reduce_mean(test_tpu, axis=0)
    iter_num_float = tf.cast(iter_num, dtype=dtype)
    return (iter_num + NUM_TPU_CORES,
            (res * iter_num_float + test_tpu)
            / (iter_num_float + NUM_TPU_CORES))
  test_tpu_res = tf.while_loop(cond, body,
                               (0, tf.zeros([NUM_SWAPS], dtype=dtype)))

In [16]:
def get_session():
  def _get_tpu_setup():
    tpu_cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
        tpu="node-1", zone="us-central1-a")
    cluster_def = tpu_cluster_resolver.cluster_spec().as_cluster_def()
    tpu_master_grpc_path = tpu_cluster_resolver.get_master()
    return cluster_def, tpu_master_grpc_path

  cluster_def, tpu_master_grpc_path = _get_tpu_setup()
  config = tf.compat.v1.ConfigProto(
      allow_soft_placement=True,
      isolate_session_state=True,
      cluster_def=cluster_def)
  return tf.compat.v1.Session(tpu_master_grpc_path, config=config)

In [17]:
with g.as_default():
  # If running on a TPU with > 8 cores:
  sess_tpu = get_session()
  # If running on a small TPU:
  # internal_ip = "10.107.181.26"
  # sess_tpu = tf.compat.v1.Session("grpc://{}:8470".format(internal_ip))
  print(sess_tpu.run(tf.compat.v1.tpu.initialize_system()))

b'\n\x04\x04\x04\x01\x02\x10\x04\x18\x08"\x80\x01\x00\x00\x00\x00\x00\x00\x00\x01\x01\x00\x00\x00\x01\x00\x00\x01\x00\x01\x00\x00\x00\x01\x00\x01\x01\x01\x00\x00\x01\x01\x00\x01\x02\x02\x00\x00\x02\x02\x00\x01\x03\x02\x00\x00\x03\x02\x00\x01\x02\x03\x00\x00\x02\x03\x00\x01\x03\x03\x00\x00\x03\x03\x00\x01\x02\x00\x00\x00\x02\x00\x00\x01\x03\x00\x00\x00\x03\x00\x00\x01\x02\x01\x00\x00\x02\x01\x00\x01\x03\x01\x00\x00\x03\x01\x00\x01\x00\x02\x00\x00\x00\x02\x00\x01\x01\x02\x00\x00\x01\x02\x00\x01\x00\x03\x00\x00\x00\x03\x00\x01\x01\x03\x00\x00\x01\x03\x00\x01'


In [19]:
# Time after optimization (run the cell twice)
t = time.time()
with g.as_default():
  sess_tpu.run(test_tpu_res, feed_dict={"days_tpu:0": NUM_TPU_CORES * [9],
                                        "months_tpu:0": NUM_TPU_CORES * [9],
                                        "years_tpu:0": NUM_TPU_CORES * [2015]})
print("Wall time (TPU 32 cores, 20k scenarios): ", time.time() - t)

Wall time (TPU 32 cores, 20k scenarios):  3.1102778911590576


In [116]:
# Time after optimization (run the cell twice)
t = time.time()
with g.as_default():
  sess_tpu.run(test_tpu_res, feed_dict={"days_tpu:0": NUM_TPU_CORES * [9],
                                        "months_tpu:0": NUM_TPU_CORES * [9],
                                        "years_tpu:0": NUM_TPU_CORES * [2015]})
print("Wall time (TPU 8 cores, 1k scenarios): ", time.time() - t)

Wall time:  0.29540467262268066
